## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/ds_salaries.csv"
file_path = "/FileStore/shared_uploads/a.shanker.srivastava@accenture.com/sample_data.txt"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df1=spark.read.text(file_path)

display(df1)

value
"A Precis is a summary of a story. The gist of the passage is written in as few words as possible. The reader should understand the idea expressed in the first reading by only reading the Precis. So it should have all the essential points. It is an exercise of comprehension. It must always be shorter than the original passage. Many people read carelessly and fail to understand the meaning of the passage for Precis Writing. So, it helps them to pay attention to the reading as no one writes a Precis without reading it carefully."
"Summarizing teaches one to read with concentration. It improves the overall Writing skills too. It allows one to express their thoughts clearly, concisely, and effectively. You learn to choose your words carefully and in a logical manner."
"In this article, students will be learning about what is Precis Writing, the Importance of Precis Writing, Writing Precis, Steps for Precis Writing, Features of a Good Precis, Dos and Don’ts for Precis Writing, Characteristics of Precis Elements of Precis along with respective examples."
Students can go through the official website of Vedantu to get a thorough understanding of various topics related to the English language.
The Precis Writing format has some essential points that you should take care of while writing it. The main important point for Writing a Precis is that you should read the passage carefully. Reading helps you to understand the gist of the passage and write it down more concisely. The main message and theme of the passage should be conveyed through your Precis.
"Also, it is essential to note down that the general idea of the passage for Precis Writing should not be forgotten. The key points should be written in the Precis. The language of the Precis should be focused on. Precis Writing has no fixed pattern. However, we need to follow a particular procedure for the Precis to be more effective. The right diction will help you write a good Precis."


In [0]:
from pyspark.sql.functions import split,explode,lower,upper,max,count
res=df1.withColumn("splt",explode(split(df1.value," ")))
res_df=res.withColumn("low",lower(res.splt)).drop(res.splt)
res_df=res_df.groupBy(res_df.low).count().sort('count', ascending=False)
res_df.show()

+---------+-----+
|      low|count|
+---------+-----+
|      the|   30|
|   precis|   15|
|       of|   15|
|       to|   12|
|        a|   10|
|       it|    9|
|   should|    8|
|  writing|    7|
|       be|    7|
|      for|    6|
|       is|    6|
|  passage|    6|
|      and|    6|
|  reading|    5|
|      you|    5|
|       in|    5|
|  precis.|    4|
| writing,|    4|
|essential|    3|
|      one|    3|
+---------+-----+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import col, count,desc,collect_set,explode,lit,split,element_at,when
from pyspark.sql.types import *
df.withColumn("expt", when(df.salary<80000,df.salary).otherwise("Ameer")).show(10)

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+-----+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size| expt|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+-----+
|     2023|              SE|             FT|Principal Data Sc...| 80000|            EUR|        85847|                ES|         100|              ES|           L|Ameer|
|     2023|              MI|             CT|         ML Engineer| 30000|            USD|        30000|                US|         100|              US|           S|30000|
|     2023|              MI|             CT|         ML Engineer| 25500|            USD|        25500|                US|         100|           

In [0]:
res_df=df.groupBy(df.job_title).agg(count(df.job_title).alias("emp_count")).orderBy(desc("emp_count"))
res_df.show(5)

+--------------------+---------+
|           job_title|emp_count|
+--------------------+---------+
|       Data Engineer|     1040|
|      Data Scientist|      840|
|        Data Analyst|      612|
|Machine Learning ...|      289|
|  Analytics Engineer|      103|
+--------------------+---------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import collect_list,size
res_df=df.groupBy(df.job_title).agg(collect_list(df.employment_type).alias("emp_list"))
#res_df.show()


#res_df=res_df.withColumn("title_count",res_df.groupby(res_df.job_title).agg(count(res_df.title)))
#res_df.show()
#df.filter(df.job_title=="3D Computer Vision Researcher").select(df.employment_type).distinct().show()
#
#res_df.withColumn("First Name",split(df.job_title," ")[0]).withColumn("Last Name",split(df.job_title," ")).show(10)
#res_df.withColumn("exploded_df",explode(res_df.emp_list)).show()

#new_df=res_df.withColumn("Sentence",lit("The random sentence Generator contains 1000+ random sentence created specifically for this free writing tool and found nowhere else.")).limit(1)
#new_df.withColumn("exploded_column", explode(split(new_df.Sentence," "))).groupBy('exploded_column').count().show()

#new_df.show()

+--------------------+--------------------+-----------+
|           job_title|            emp_list|title_count|
+--------------------+--------------------+-----------+
|3D Computer Visio...|    [FT, FT, FT, PT]|          4|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|          6|
|        Data Modeler|            [FT, FT]|          2|
| Data Scientist Lead|            [FT, FT]|          2|
|Principal Data Ar...|                [FT]|          1|
|Head of Machine L...|                [FT]|          1|
|Machine Learning ...|[FT, FT, FT, FT, ...|         10|
|Data Analytics Sp...|            [FT, FT]|          2|
|     Data Specialist|[FT, FT, FT, FT, ...|         14|
|Data Operations E...|[FT, FT, FT, FT, ...|         10|
|Deep Learning Res...|                [FT]|          1|
| Data Analytics Lead|            [FT, FT]|          2|
|  Power BI Developer|                [FT]|          1|
|Machine Learning ...|[FT, FT, FT, FT, ...|         26|
|   Lead Data Analyst|[FT, FT, FT, FT, FT]|     

In [0]:
res_df=res_df.withColumn("title",explode(res_df.emp_list))
res_df.show()

+--------------------+--------------------+-----+
|           job_title|            emp_list|title|
+--------------------+--------------------+-----+
|3D Computer Visio...|    [FT, FT, FT, PT]|   FT|
|3D Computer Visio...|    [FT, FT, FT, PT]|   FT|
|3D Computer Visio...|    [FT, FT, FT, PT]|   FT|
|3D Computer Visio...|    [FT, FT, FT, PT]|   PT|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|
|        Data Modeler|            [FT, FT]|   FT|
|        Data Modeler|            [FT, FT]|   FT|
| Data Scientist Lead|            [FT, FT]|   FT|
| Data Scientist Lead|            [FT, FT]|   FT|
|Principal Data Ar...|                [FT]|   FT|
|Head of Machine L...|                [FT]|   FT|
|Machine Learning ...|[FT, FT, FT, FT, ...|   FT|


In [0]:
total=res_df.withColumn("title_count",size(res_df.emp_list))
total.show()

+--------------------+--------------------+-----+-----------+
|           job_title|            emp_list|title|title_count|
+--------------------+--------------------+-----+-----------+
|3D Computer Visio...|    [FT, FT, FT, PT]|   FT|          4|
|3D Computer Visio...|    [FT, FT, FT, PT]|   FT|          4|
|3D Computer Visio...|    [FT, FT, FT, PT]|   FT|          4|
|3D Computer Visio...|    [FT, FT, FT, PT]|   PT|          4|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|          6|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|          6|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|          6|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|          6|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|          6|
|  Lead Data Engineer|[FT, FT, FT, FT, ...|   FT|          6|
|        Data Modeler|            [FT, FT]|   FT|          2|
|        Data Modeler|            [FT, FT]|   FT|          2|
| Data Scientist Lead|            [FT, FT]|   FT|          2|
| Data S

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "ds_salaries_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)